# **Responsible** AI is about an ownership mentality of what AI does in products, what it does with people, what it does _to_ people, and knowing that everything ultimately impacts our earth.

## 💙 Responsible AI Principles at Microsoft

| Liberal Arts | Engineering |
|---|---|
| Fairness | Reliability & Safety |
| Inclusiveness | Privacy & Security |
| Accountability | Transparency |

## 📅 The Copilot Stack in March 2023, by Kevin Scott CTO and EVP of AI @ Microsoft

![](assets/copilotstack.png)

## 🔖 The TL;DR of the Copilot Stack @Microsoft

![](assets/copilotstackwazure.png)

Let's show how that commitment goes deep into the Copilot Stack by popping into Azure to [deploy a model to see how safety gets baked in wherever is technically possible](https://azure.microsoft.com/en-us/products/ai-services/ai-content-safety/). And ICYMI there's a popular 📺 course on [**"AI UX for Developers"**](https://www.linkedin.com/learning/ux-for-ai-design-practices-for-ai-developers) to learn about Responsible AI at the upper levels of the Copilot Stack.

## 🔥 We warm up a kernel

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.1.0"

In [ ]:
// Load settings
#!import config/Settings.cs 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

In [ ]:
using System;
using System.ComponentModel;
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Kernel = Microsoft.SemanticKernel.Kernel;

// Create a kernel with OpenAI chat completion
IKernelBuilder kernelBuilder = Kernel.CreateBuilder();
kernelBuilder.AddOpenAIChatCompletion(
        modelId: model,
        apiKey: apiKey);

Kernel kernel = kernelBuilder.Build();

## 🚚 We can add a prompt rendering handle to intercept prompts coming into SK

In [ ]:
#pragma warning disable SKEXP0004

// Handler which is called before a prompt is rendered
void MyRenderingHandler(object? sender, PromptRenderingEventArgs e)
{
    if (e.Arguments.ContainsName("card_number"))
    {
        e.Arguments["card_number"] = "**** **** **** ****";
    }
}
kernel.PromptRendering += MyRenderingHandler;


## 🚚 We can add extra verbiage to every prompt with the prompt rendered handle

In [ ]:
#pragma warning disable SKEXP0004

// Handler which is called after a prompt is rendered
void MyRenderedHandler(object? sender, PromptRenderedEventArgs e)
{
    e.RenderedPrompt += " NO SEXISM, RACISM OR OTHER BIAS/BIGOTRY";

    Console.WriteLine($"Complete rendered prompt:\n>> {e.RenderedPrompt}");
}
kernel.PromptRendered += MyRenderedHandler;

## ❤️ This all means we can consciously tune everything going into the LLM as responsibly as we can

Let's simulate a credit card number is going into the LLM as input data. We're able to monitor if an argument is submitting info, or we can scan the entire prompt to sanitize it.

In [ ]:
KernelArguments arguments = new() { { "card_number", "4444 3333 2222 1111" } };
Console.WriteLine(await kernel.InvokePromptAsync("Tell me some useful information about this credit card number {{$card_number}}?", arguments));

Alternatively, let's say something terribly in appropriate. Note that the model itself has been trained to stay within appropriate conversation norms.

In [ ]:
Console.WriteLine(await kernel.InvokePromptAsync("I don't think that men can cook."));